## ECMWF open data: Operational Real-Time Forecasts

Documentation: 

https://confluence.ecmwf.int/display/DAC/ECMWF+open+data%3A+real-time+forecasts+from+IFS+and+AIFS

https://github.com/ecmwf/ecmwf-opendat

https://codes.ecmwf.int/grib/param-db/a

In [62]:
from ecmwf.opendata import Client
import ecmwf.data as ecdata
import xarray as xr
import pandas as pd
import numpy as np
import os
from datetime import timedelta,datetime

#### Descarga del archivo GRIB desde ECMWF

In [2]:
client = Client(
    source="ecmwf",                # name of server to contact or a fully qualified URL
    model="ifs",                   # ifs for the physics-driven model and aifs for the data-driven model (currently experimental)    
    resol="0p25",                  # 0p25 for 0.25 degree resolution, 32Km. The paid version has 9Km resoluction  
    preserve_request_order=False,
    infer_stream_keyword=True,
)

In [3]:
# Get the current date
current_date = datetime.now().strftime('%Y%m%d')

# Define the base path
base_path = 'D:\\Documents\\MMA\\1.0 Tesis\\Datos\\ECMWF\\'

# Create the filename with the current date
filename = f'{base_path}ecmwf_forecast_{current_date}.grib'

#filename = 'D:\\Documents\\MMA\\1.0 Tesis\\Datos\\ECMWF\\ecmwf_forecast.grib'
filename

'D:\\Documents\\MMA\\1.0 Tesis\\Datos\\ECMWF\\ecmwf_forecast_20240712.grib'

In [4]:
# Parametros Disponibles

#100u: 100m U wind component (m/s)
#100v: 100m V wind component (m/s)
#10u: 10m U wind component (m/s)
#10v: 10m V wind component (m/s)
#2d: 2-meter dewpoint temperature (K)
#2t: 2-meter temperature (K)
#asn: Accumulated snow depth (m of water equivalent)
#cape: Convective available potential energy (J/kg)
#d: Divergence (s^-1)
#gh: Geopotential height (m)
#lsm: Land-sea mask (0 to 1)
#mn2t6: Minimum temperature at 2 meters during the last 6 hours (K)
#msl: Mean sea-level pressure (Pa)
#mx2t6: Maximum temperature at 2 meters during the last 6 hours (K)
#q: Specific humidity (kg/kg)
#r: Relative humidity (%)
#ro: Soil moisture content (fraction)
#skt: Skin temperature (K)
#sp: Surface pressure (Pa)
#ssr: Surface solar radiation (W/m^2)
#ssrd: Downward surface solar radiation (W/m^2)
#st: Soil temperature (K)
#stl2, stl3, stl4: Soil temperature levels 2, 3, and 4 (K)
#str: Soil temperature range (K)
#strd: Downward surface thermal radiation (W/m^2)
#swvl1, swvl2, swvl3, swvl4: Volumetric soil water layers 1, 2, 3, and 4 (m^3/m^3)
#t: Temperature (K)
#tcwv: Total column water vapor (kg/m^2)
#tp: Total precipitation (m)
#ttr: Top thermal radiation (W/m^2)
#u: U wind component (m/s)
#v: V wind component (m/s)
#vo: Vorticity (s^-1)
#w: Vertical velocity (Pa/s)

In [5]:
parameters = ['100u', '100v','2t','sp','q'] #100m U V wind components m s-1, 2t: 2m Temperature (K), , sp:Surface pressure	Pa, #q: Specific humidity (kg/kg)

request = {
    "date": 0,                            #The date and time parameters refer to the starting time of the forecast 
    "time": 0,
    "type": "fc",                         #HRES: High Resolution Forecast. fc: Forecast.   
    "stream": "oper",                     # Operational stream. oper: Atmospheric fields from HRES - 00 UTC and 12 UTC.
    "step": list(range(0, 145, 3)) + list(range(150, 241, 6)),     # HRES 00 and 12 - 0 to 144 by 3hs, 144 to 240 by 6hs 
    "param": parameters # ['100u', '100v','2t','sp']   # 100m U V wind components m s-1, 2 metre temperature	K, sp:Surface pressure	Pa, 
}


In [6]:
client.retrieve(
    request=request,
    target = filename     # Output file name
)

set() ['fc']
set() ['0', '102', '105', '108', '111', '114', '117', '12', '120', '123', '126', '129', '132', '135', '138', '141', '144', '15', '150', '156', '162', '168', '174', '18', '180', '186', '192', '198', '204', '21', '210', '216', '222', '228', '234', '24', '240', '27', '3', '30', '33', '36', '39', '42', '45', '48', '51', '54', '57', '6', '60', '63', '66', '69', '72', '75', '78', '81', '84', '87', '9', '90', '93', '96', '99']
set() ['100u', '100v', '10u', '10v', '2d', '2t', 'asn', 'cape', 'd', 'gh', 'lsm', 'mn2t6', 'msl', 'mx2t6', 'q', 'r', 'ro', 'skt', 'sp', 'ssr', 'ssrd', 'st', 'stl2', 'stl3', 'stl4', 'str', 'strd', 'swvl1', 'swvl2', 'swvl3', 'swvl4', 't', 'tcwv', 'tp', 'ttr', 'u', 'v', 'vo', 'w']


<multiple>:   0%|          | 0.00/758M [00:00<?, ?B/s]

### Reading the file and processing the data

In [63]:
filename = 'D:\\Documents\\MMA\\1.0 Tesis\\Datos\\ECMWF\\ecmwf_forecast_20240602.grib'
#filename = 'D:\\Documents\\MMA\\1.0 Tesis\\Datos\\ECMWF\\ecmwf_forecast_20240712.grib'

In [64]:
data = ecdata.read(filename)

In [65]:
data.describe()

parameter,typeOfLevel,level,date,time,step,number,paramId,class,stream,type,experimentVersionNumber
100u,heightAboveGround,100,20240602,0,"0,3,...",None,228246,od,oper,fc,0001
100v,heightAboveGround,100,20240602,0,"0,3,...",None,228247,od,oper,fc,0001
2t,heightAboveGround,2,20240602,0,"0,3,...",None,167,od,oper,fc,0001
q,isobaricInhPa,"50,100,...",20240602,0,"0,3,...",None,133,od,oper,fc,0001
sp,surface,0,20240602,0,"0,3,...",None,134,od,oper,fc,0001


#### GRIB2 file to xarray

In [66]:
# Open the GRIB2 file using xarray with the cfgrib engine
ds_0m = xr.open_dataset(filename, engine='cfgrib',filter_by_keys={'typeOfLevel': 'surface', 'level': 0})
ds_2m = xr.open_dataset(filename, engine='cfgrib',filter_by_keys={'typeOfLevel': 'heightAboveGround', 'level': 2})
ds_100m = xr.open_dataset(filename, engine='cfgrib',filter_by_keys={'typeOfLevel': 'heightAboveGround', 'level': 100})
ds_1000hpa = xr.open_dataset(filename, engine='cfgrib',filter_by_keys={'typeOfLevel': 'isobaricInhPa', 'level': 1000})

In [67]:
# Remove the conflicting coordinate variable before concatenating
ds_0m = ds_0m.drop_vars('surface')
ds_2m = ds_2m.drop_vars('heightAboveGround')
ds_100m = ds_100m.drop_vars('heightAboveGround')
ds_1000hpa = ds_1000hpa.drop_vars('isobaricInhPa')


In [68]:
# Specify the latitude and longitude for La Castellana Bahia Blanca
lat_location = -38.5
lon_location = -62.75

##### Wind Data Extraction

In [69]:
# Extract data for the specified location and time range
ds_100m = ds_100m.sel(latitude=lat_location, longitude=lon_location, method='nearest')

In [70]:
# Convert xarray dataset to pandas DataFrame
df_100m = ds_100m.to_dataframe().reset_index().loc[:,['valid_time','v100','u100']]

In [71]:
# Rename the "valid_time" column to "FechaHora" and convert values to datetime
df_100m = df_100m.rename(columns={'valid_time': 'FechaHora'})
df_100m['FechaHora'] = pd.to_datetime(df_100m['FechaHora'])

##### Wind Speed Formula:

The wind speed (WS) can be calculated from the zonal (u) and meridional (v) wind components using the Pythagorean theorem:

WS = sqrt(u^2 + v^2)

Where:
- WS is the wind speed,
- u is the zonal (east-west) component of the wind,
- v is the meridional (north-south) component of the wind.

This formula computes the magnitude of the wind vector, representing the total speed of the wind regardless of its directio

##### Wind Direction Formula: 

Can be calculated using trigonometric functions arctan2 from the u and v components:

If the wind components are given in a Cartesian coordinate system (u = east-west, v = north-south):

Use the arctan2 function to calculate the wind direction

    # Calculate the angle in radians
     theta = np.arctan2(v, u)
    
    # Convert the angle from radians to degrees
    theta_deg = np.degrees(theta)
    
    # Adjust wind direction to be between 0° and 360°
    wd_deg = (270 - theta_deg) % 360 to cardinal directions).



In [72]:
# Calculate wind speed from u and v components (magnitude of the vector)
df_100m['ws100'] = np.sqrt(df_100m['u100']**2 + df_100m['v100']**2)

In [73]:
# Calculate wind dir from u and v components (magnitude of the vector)

# Calculate the angle in radians
theta = np.arctan2(df_100m['v100'], df_100m['u100'])

# Convert the angle from radians to degrees
theta_deg = np.degrees(theta)

df_100m['dir100'] = (270 - theta_deg) % 360

In [74]:
# Drop 'v100' and 'u100' columns
df_100m = df_100m.drop(columns=['v100', 'u100'])

#df_100m.head()

##### Temperature Extraction

In [75]:
# Extract data for the specified location and time range
ds_2m = ds_2m.sel(latitude=lat_location, longitude=lon_location, method='nearest')

In [76]:
# Convert xarray dataset to pandas DataFrame
df_2m = ds_2m.to_dataframe().reset_index().loc[:,['valid_time','t2m']]

In [77]:
# Rename the "valid_time" column to "FechaHora" and convert values to datetime
df_2m = df_2m.rename(columns={'valid_time': 'FechaHora'})
df_2m['FechaHora'] = pd.to_datetime(df_2m['FechaHora'])

In [78]:
# Convert temperature from Kelvin to Celsius
#df_2m['t2m'] -= 273.15

##### Surface Presure Extraction

In [79]:
# Extract data for the specified location and time range
ds_0m = ds_0m.sel(latitude=lat_location, longitude=lon_location, method='nearest')

In [80]:
# Convert xarray dataset to pandas DataFrame
df_0m = ds_0m.to_dataframe().reset_index().loc[:,['valid_time','sp']]

In [81]:
# Rename the "valid_time" column to "FechaHora" and convert values to datetime
df_0m = df_0m.rename(columns={'valid_time': 'FechaHora'})
df_0m['FechaHora'] = pd.to_datetime(df_0m['FechaHora'])

In [82]:
# Convert pressure from Pa to hPa
#df_0m['sp'] /= 100

##### Specific Humidity Extraction

In [83]:
# Extract data for the specified location and time range
ds_1000hpa = ds_1000hpa.sel(latitude=lat_location, longitude=lon_location, method='nearest')

In [84]:
# Convert xarray dataset to pandas DataFrame
ds_1000hpa = ds_1000hpa.to_dataframe().reset_index().loc[:,['valid_time','q']]

In [85]:
# Rename the "valid_time" column to "FechaHora" and convert values to datetime
ds_1000hpa = ds_1000hpa.rename(columns={'valid_time': 'FechaHora'})
ds_1000hpa['FechaHora'] = pd.to_datetime(ds_1000hpa['FechaHora'])

##### Merge DataFrames

In [86]:
# Merge df_100m and df_2m on 'FechaHora'
df_merged = pd.merge(df_100m, df_2m, on='FechaHora')

# Merge the result with df_0m on 'FechaHora'
df_merged = pd.merge(df_merged, df_0m, on='FechaHora')

# Merge the result with ds_1000hpa on 'FechaHora'
df_ecmwf_fc = pd.merge(df_merged, ds_1000hpa, on='FechaHora')

#### Air density calculation at surface level

El ECMWF open dataset ERA5 hourly data on single levels, no cuanta con la densidad del aire la calulo con las variables de temperatura y presión atm a nivel superficie y la humedad especifica la obtengo del dataset ERA5 hourly data on pressure levels al nivel isobarico de 1000hpa que es la presión mas aproximada a nivel de superficie.

La densidad del aire se utiliza para el calculo de la fuerza de empuje en el rotor 

In [87]:
# Constants
R_dry = 287.058  # Specific gas constant for dry air, J/(kg·K)
R_vapor = 461.495  # Specific gas constant for water vapor, J/(kg·K)
epsilon = R_dry / R_vapor

# Given data (example values, replace with actual data)
T_2m = df_ecmwf_fc['t2m']  # Temperature at 2 meters (K)
P_surface = df_ecmwf_fc['sp']  # Surface pressure (Pa)
q_1000hPa = df_ecmwf_fc['q']  # Specific humidity at 1000 hPa (kg/kg)


# Calculate air density
e = q_1000hPa * P_surface / (epsilon + q_1000hPa)  # Partial pressure of water vapor (Pa)
P_dry = P_surface - e  # Partial pressure of dry air (Pa)
rho_dry = P_dry / (R_dry * T_2m)  # Density of dry air (kg/m³)
rho_vapor = e / (R_vapor * T_2m)  # Density of water vapor (kg/m³)

df_ecmwf_fc['air_density'] = rho_dry + rho_vapor  # Total air density (kg/m³)

#print("Air density:", df_ecmwf_fc['air_density'], "kg/m³")

##### Calculate Relation Thrust Force T (Fuerza de Empuje) / Coefficient Ct (Coeficiente de Empuje) 

En el contexto de las turbinas eólicas, la fuerza de empuje es la fuerza aerodinámica que el viento ejerce sobre las palas del rotor de una turbina.
Esta fuerza afecta tanto el rendimiento de la turbina como la distribución de cargas en su estructura, lo que es crucial para el diseño y la operación eficientes.

T / Ct = 0.5 * air_density * Area_rotor * wind_speed^2

Ct= es un coeficiente dado por el fabricante

In [88]:
# rotor diameter in meters 
D = 125 # Turbine Acciona AW 125/3150

# Calculate rotor swept area (m²)
A = np.pi * (D / 2) ** 2

# Calculate relation between Thrust Force / Thrust Coefficient C_T
df_ecmwf_fc['rel_Tct'] = 0.5 * df_ecmwf_fc['air_density'] * df_ecmwf_fc['ws100']**2

In [89]:
df_ecmwf_fc.describe()

,FechaHora,ws100,dir100,t2m,sp,q,air_density,rel_Tct
count,65,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000
mean,2024-06-06 06:16:36.923076864,7.473932,229.971542,283.043274,100230.625000,0.005987,1.229437,41.150700
min,2024-06-02 00:00:00,0.769684,1.235413,274.136597,98776.156250,0.003253,1.178488,0.354163
25%,2024-06-04 00:00:00,5.023561,119.810135,280.952271,99831.218750,0.004515,1.212570,16.148857
50%,2024-06-06 00:00:00,7.618662,272.558502,283.072601,100174.609375,0.005756,1.233446,35.132858
75%,2024-06-08 00:00:00,9.733680,345.113708,285.153351,100540.593750,0.006841,1.241058,58.518669
max,2024-06-12 00:00:00,15.220150,358.750427,291.969482,101598.359375,0.010211,1.287602,144.640686
std,NaN,3.336324,126.333611,3.574959,610.574890,0.001685,0.022138,32.895782


##### Fill Hourly Gaps

In [90]:
# Subtract 3 hours from the 'time' column to convert to GMT(-3)
df_ecmwf_fc['FechaHora'] -= timedelta(hours=3)

# Find the minimum date where time is 00:00:00
min_date = df_ecmwf_fc[df_ecmwf_fc['FechaHora'].dt.time == pd.to_datetime('00:00:00').time()]['FechaHora'].min()

# Filter the DataFrame to keep records from this date onward
df_ecmwf_fc = df_ecmwf_fc[df_ecmwf_fc['FechaHora'] >= min_date]


In [91]:
# Convert 'FechaHora' to datetime
df_ecmwf_fc['FechaHora'] = pd.to_datetime(df_ecmwf_fc['FechaHora'])

In [92]:
# Set 'FechaHora' as the index
df_ecmwf_fc.set_index('FechaHora', inplace=True)

In [93]:
# Resample the dataframe to hourly frequency
df_ecmwf_fc = df_ecmwf_fc.resample('h').asfreq()


In [94]:
# Linear interpolation
df_ecmwf_fc = df_ecmwf_fc.interpolate(method='linear')

In [95]:
df_ecmwf_fc.head()

,ws100,dir100,t2m,sp,q,air_density,rel_Tct
FechaHora,,,,,,,
2024-06-02 00:00:00,2.440190,322.811096,279.690918,99959.031250,0.004624,1.241541,3.696394
2024-06-02 01:00:00,3.847449,308.847656,279.953033,99974.757812,0.004267,1.240842,11.632383
2024-06-02 02:00:00,5.254708,294.884186,280.215149,99990.492188,0.003909,1.240142,19.568371
2024-06-02 03:00:00,6.661968,280.920746,280.477264,100006.218750,0.003552,1.239443,27.504360
2024-06-02 04:00:00,6.919897,277.907471,280.482483,100032.070312,0.003453,1.239815,29.768076


In [96]:
# Convert temperature from Kelvin to Celsius
df_ecmwf_fc['t2m'] -= 273.15

In [97]:
# Round 2 decimal
df_ecmwf_fc = df_ecmwf_fc.astype(float).round(3)

In [98]:
df_ecmwf_fc.describe()

,ws100,dir100,t2m,sp,q,air_density,rel_Tct
count,238.000000,238.000000,238.000000,238.000000,238.000000,238.000000,238.000000
mean,7.183227,212.413622,10.244803,100163.523160,0.006269,1.226933,38.057542
std,3.118924,121.170449,3.338307,620.317225,0.001801,0.021397,30.291450
min,0.770000,1.235000,0.987000,98776.156000,0.003000,1.178000,0.354000
25%,4.573000,112.313250,7.888500,99790.443000,0.005000,1.211000,13.319250
50%,7.384000,234.007000,10.042500,100139.926000,0.006000,1.227500,33.491500
75%,9.003500,336.568000,12.507000,100496.306500,0.007000,1.240000,50.993250
max,15.220000,358.750000,18.819000,101598.359000,0.010000,1.288000,144.641000


In [99]:
# Select the main columns
df_ecmwf_fc_r = df_ecmwf_fc[['ws100', 'dir100','t2m','air_density','rel_Tct']]

In [100]:
# Rename the columns
df_ecmwf_fc_r = df_ecmwf_fc_r.rename(columns={
    'ws100': 'ws100_ecmwf',
    'dir100': 'dir100_ecmwf',
    't2m': 'temp_ecmwf',
    'air_density': 'airden_ecmwf',
    'rel_Tct': 'reltct_ecmwf'
})

In [101]:
df_ecmwf_fc_r.reset_index(inplace=True)

In [102]:
df_ecmwf_fc_r.head()

,FechaHora,ws100_ecmwf,dir100_ecmwf,temp_ecmwf,airden_ecmwf,reltct_ecmwf
0,2024-06-02 00:00:00,2.440,322.811,6.541,1.242,3.696
1,2024-06-02 01:00:00,3.847,308.848,6.803,1.241,11.632
2,2024-06-02 02:00:00,5.255,294.884,7.065,1.240,19.568
3,2024-06-02 03:00:00,6.662,280.921,7.327,1.239,27.504
4,2024-06-02 04:00:00,6.920,277.907,7.332,1.240,29.768


In [103]:
min_date = min_date.strftime('%Y-%m-%d')

In [104]:
df_ecmwf_fc_r.to_csv(f'D:\\Documents\\MMA\\1.0 Tesis\\Datos\\ECMWF\\ds_ecmwf_fc_{min_date}.csv', sep=';', index=False, decimal=',')

## Historical ERA5 Reanalisis Wheather data

link to downloas the files single level:
https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form

link to downloas the files presure level:
https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=form


ERA5 is the fifth generation ECMWF reanalysis for the global climate and weather for the past 8 decades. Data is available from 1940 onwards. ERA5 replaces the ERA-Interim reanalysis.

Reanalysis combines model data with observations from across the world into a globally complete and consistent dataset using the laws of physics. This principle, called data assimilation, is based on the method used by numerical weather prediction centres, where every so many hours (12 hours at ECMWF) a previous forecast is combined with newly available observations in an optimal way to produce a new best estimate of the state of the atmosphere, called analysis, from which an updated, improved forecast is issued. Reanalysis works in the same way, but at reduced resolution to allow for the provision of a dataset spanning back several decades. Reanalysis does not have the constraint of issuing timely forecasts, so there is more time to collect observations, and when going further back in time, to allow for the ingestion of improved versions of the original observations, which all benefit the quality of the reanalysis product.

ERA5 provides hourly estimates for a large number of atmospheric, ocean-wave and land-surface quantities. An uncertainty estimate is sampled by an underlying 10-member ensemble at three-hourly intervals. Ensemble mean and spread have been pre-computed for convenience. Such uncertainty estimates are closely related to the information content of the available observing system which has evolved considerably over time. They also indicate flow-dependent sensitive areas. To facilitate many climate applications, monthly-mean averages have been pre-calculated too, though monthly means are not available for the ensemble mean and spread.

ERA5 is updated daily with a latency of about 5 days. In case that serious flaws are detected in this early release (called ERA5T), this data could be different from the final release 2 to 3 months later. In case that this occurs users are notified.

The data set presented here is a regridded subset of the full ERA5 data set on native resolution. It is online on spinning disk, which should ensure fast and easy access. It should satisfy the requirements for most common applications.

An overview of all ERA5 datasets can be found in this article. Information on access to ERA5 data on native resolution is provided in these guidelines.

Data has been regridded to a regular lat-lon grid of 0.25 degrees for the reanalysis and 0.5 degrees for the uncertainty estimate (0.5 and 1 degree respectively for ocean waves). There are four main sub sets: hourly and monthly products, both on pressure levels (upper air fields) and single levels (atmospheric, ocean-wave and land surface quantities).

The present entry is "ERA5 hourly data on single levels from 1940 to present".


#####  Carga los archivos de ERA5 des archivos locales

In [50]:

# Specify the directory where the NetCDF files are stored
directory = 'D:\\Documents\\MMA\\1.0 Tesis\\Datos\\ERA5'

# Initialize a dictionary to store DataFrames for each year
yearly_data_frames = {}

# Specify the latitude and longitude for La Castellana Bahia Blanca
lat_location = -38.5
lon_location = -62.75

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".nc"):  # Assuming all files are NetCDF format
        filepath = os.path.join(directory, filename)
        print(filepath)
        # Extract the year from the filename
        year = int(filename.split('-')[-1].split('.')[0])
        # Open the NetCDF file using xarray
        data = xr.open_dataset(filepath)
        # Select the desired variables ('u100', 'v100', 't2m', 'sp','q')
        for var_name in ['u100', 'v100', 't2m', 'sp','q']:
            # Check if the variable exists in the dataset
            if var_name in data:
                # Filter the variable data based on the latitude and longitude location
                filtered_data = data[var_name].sel(latitude=lat_location, longitude=lon_location, method='nearest')
                # Convert the filtered DataArray to a DataFrame
                df= filtered_data.to_dataframe(name=var_name)
                # Add the DataFrame to the dictionary with the year as the key
                if year not in yearly_data_frames:
                    yearly_data_frames[year] = df
                else:
                    yearly_data_frames[year] = pd.concat([yearly_data_frames[year], df], axis=1)

# Concatenate all DataFrames in the dictionary along the index (rows) to combine the data for each year
df_histo_era5 = pd.concat(yearly_data_frames.values())

# Drop repeated longitude and latitude columns
df_histo_era5 = df_histo_era5.loc[:,~df_histo_era5.columns.duplicated()]

# For the current year the records has in the index the number of experiment with values 1 or 5, records with more than 2 month has the values with 
# exper = 1 and null for exper=5 and in the other way around for the last 2 month.
df_histo_era5.dropna(inplace=True)

# lambda function to map the index to the first element if it's a tuple
df_histo_era5.index = list(map(lambda idx: idx[0] if isinstance(idx, tuple) else idx, df_histo_era5.index))

# Convert the index to DateTimeIndex to ensure consistency
df_histo_era5.index = pd.to_datetime(df_histo_era5.index)

#Reset Index
df_histo_era5.reset_index(inplace=True)

# Rename the index column to 'FechaHora'
df_histo_era5 = df_histo_era5.rename(columns={'index': 'FechaHora'})

D:\Documents\MMA\1.0 Tesis\Datos\ERA5\adaptor.mars.internal-2019.nc
D:\Documents\MMA\1.0 Tesis\Datos\ERA5\adaptor.mars.internal-2020.nc
D:\Documents\MMA\1.0 Tesis\Datos\ERA5\adaptor.mars.internal-2021.nc
D:\Documents\MMA\1.0 Tesis\Datos\ERA5\adaptor.mars.internal-2022.nc
D:\Documents\MMA\1.0 Tesis\Datos\ERA5\adaptor.mars.internal-2023.nc
D:\Documents\MMA\1.0 Tesis\Datos\ERA5\adaptor.mars.internal-2024.nc
D:\Documents\MMA\1.0 Tesis\Datos\ERA5\adaptor.mars.internal.presurelevel-2019.nc
D:\Documents\MMA\1.0 Tesis\Datos\ERA5\adaptor.mars.internal.presurelevel-2020.nc
D:\Documents\MMA\1.0 Tesis\Datos\ERA5\adaptor.mars.internal.presurelevel-2021.nc
D:\Documents\MMA\1.0 Tesis\Datos\ERA5\adaptor.mars.internal.presurelevel-2022.nc
D:\Documents\MMA\1.0 Tesis\Datos\ERA5\adaptor.mars.internal.presurelevel-2023.nc
D:\Documents\MMA\1.0 Tesis\Datos\ERA5\adaptor.mars.internal.presurelevel-2024.nc


In [51]:
df_histo_era5.tail()

,FechaHora,longitude,latitude,u100,v100,t2m,sp,q
48082,2024-06-26 10:00:00,-62.75,-38.5,9.621508,-6.592616,280.469107,99932.770540,0.004039
48083,2024-06-26 11:00:00,-62.75,-38.5,10.146357,-6.095953,280.765418,99929.946203,0.003976
48084,2024-06-26 12:00:00,-62.75,-38.5,10.661417,-6.837992,280.425854,99921.250216,0.004145
48085,2024-06-26 13:00:00,-62.75,-38.5,10.052983,-7.906706,280.412298,99831.986285,0.004161
48086,2024-06-26 14:00:00,-62.75,-38.5,9.960740,-8.241757,281.987460,99826.114635,0.004166


In [52]:
# Calculate wind speed from u and v components (magnitude of the vector)
df_histo_era5['ws100'] = np.sqrt(df_histo_era5['u100']**2 + df_histo_era5['v100']**2)

# Calculate wind dir from u and v components (magnitude of the vector)

# Calculate the angle in radians
theta = np.arctan2(df_histo_era5['v100'], df_histo_era5['u100'])

# Convert the angle from radians to degrees
theta_deg = np.degrees(theta)

df_histo_era5['dir100'] = (270 - theta_deg) % 360


#### Air density calculation at surface level

In [53]:
# Constants
R_dry = 287.058  # Specific gas constant for dry air, J/(kg·K)
R_vapor = 461.495  # Specific gas constant for water vapor, J/(kg·K)
epsilon = R_dry / R_vapor

# Given data (example values, replace with actual data)
T_2m = df_histo_era5['t2m']  # Temperature at 2 meters (K)
P_surface = df_histo_era5['sp']  # Surface pressure (Pa)
q_1000hPa = df_histo_era5['q']  # Specific humidity at 1000 hPa (kg/kg)


# Calculate air density
e = q_1000hPa * P_surface / (epsilon + q_1000hPa)  # Partial pressure of water vapor (Pa)
P_dry = P_surface - e  # Partial pressure of dry air (Pa)
rho_dry = P_dry / (R_dry * T_2m)  # Density of dry air (kg/m³)
rho_vapor = e / (R_vapor * T_2m)  # Density of water vapor (kg/m³)

df_histo_era5['air_density'] = rho_dry + rho_vapor  # Total air density (kg/m³)

#print("Air density:", df_ecmwf_fc['air_density'], "kg/m³")

##### Calculate Relation Thrust Force T (Fuerza de Empuje) / Coefficient Ct (Coeficiente de Empuje) 

In [54]:
# rotor diameter in meters 
D = 125 # Turbine Acciona AW 125/3150

# Calculate rotor swept area (m²)
A = np.pi * (D / 2) ** 2

# Calculate relation between Thrust Force / Thrust Coefficient C_T
df_histo_era5['rel_Tct'] = 0.5 * df_histo_era5['air_density'] * df_histo_era5['ws100']**2

#### Convert Temperature

In [55]:
# Convert temperature from Kelvin to Celsius
df_histo_era5['t2m'] -= 273.15

##### Adjust Time & dataframe

In [56]:
# Subtract 3 hours from the 'time' column to convert to GMT(-3)
df_histo_era5['FechaHora'] -= timedelta(hours=3)

# Set 'FechaHora' as the index
df_histo_era5.set_index('FechaHora', inplace=True)

# Select the main columns
df_histo_era5_r = df_histo_era5[['ws100', 'dir100','t2m','air_density','rel_Tct']].astype(float)

# Round 2 decimal
#df_histo_era5_r  = df_histo_era5_r .round(3)

In [57]:
# Rename the columns
df_histo_era5_r = df_histo_era5_r.rename(columns={
    'ws100': 'ws100_ecmwf',
    'dir100': 'dir100_ecmwf',
    't2m': 'temp_ecmwf',
    'air_density': 'airden_ecmwf',
    'rel_Tct': 'reltct_ecmwf'
})

In [58]:
df_histo_era5_r.head()

,ws100_ecmwf,dir100_ecmwf,temp_ecmwf,airden_ecmwf,reltct_ecmwf
FechaHora,,,,,
2018-12-31 21:00:00,12.121110,357.113901,21.379415,1.169393,85.904362
2018-12-31 22:00:00,13.585819,359.681664,21.398169,1.170022,107.978115
2018-12-31 23:00:00,13.775665,0.234681,21.240099,1.170395,111.052331
2019-01-01 00:00:00,12.837879,358.451270,20.925970,1.171574,96.544244
2019-01-01 01:00:00,11.800872,354.468828,21.206610,1.170418,81.496515


In [59]:
df_histo_era5_r.reset_index(inplace=True)

In [60]:
df_histo_era5_r.to_csv('D:\\Documents\\MMA\\1.0 Tesis\\Datos\\ERA5\\ds_histo_era5.csv', sep=';', index=False, decimal=',')

## ___________________________________________________________________________________________________

## API - ERA5 Reanalisis

Copernicus Climate Change Service (C3S)

The application allows users to calculate, on demand, daily statistics for a number of variables published from ERA5 and to download the resulting data in a file.


In [ ]:
'''
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'variable': [
            '100m_u_component_of_wind', '100m_v_component_of_wind', '2m_temperature',
            'surface_pressure','specific_humidity'
        ],
        'year': '2024',
        'month': [
            '01', '02', '03',
            '04', '05', '06'
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            --38.5, -62.75, -39,
            -62,
        ],
    },
    'download.grib')
    '''

In [ ]:
#### descargado desde CDS

In [ ]:
'''
import cdsapi
import requests
 
# CDS API script to use CDS service to retrieve daily ERA5* variables and iterate over
# all months in the specified years.
 
# Requires:
# 1) the CDS API to be installed and working on your system
# 2) You have agreed to the ERA5 Licence (via the CDS web page)
# 3) Selection of required variable, daily statistic, etc
 
# Output:
# 1) separate netCDF file for chosen daily statistic/variable for each month
 
c = cdsapi.Client(timeout=300)
 
# Uncomment years as required
 
years =  [
            '1979'
#           ,'1980', '1981',
#            '1982', '1983', '1984',
#            '1985', '1986', '1987',
#            '1988', '1989', '1990',
#            '1991', '1992', '1993',
#            '1994', '1995', '1996',
#            '1997', '1998', '1999',
#            '2000', '2001', '2002',
#            '2003', '2004', '2005',
#            '2006', '2007', '2008',
#            '2009', '2010', '2011',
#            '2012', '2013', '2014',
#            '2015', '2016', '2017',
#            '2018', '2019', '2020',
#            '2021'
]
 
 
# Retrieve all months for a given year.
 
months = ['01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12']
 
# For valid keywords, see Table 2 of:
# https://datastore.copernicus-climate.eu/documents/app-c3s-daily-era5-statistics/C3S_Application-Documentation_ERA5-daily-statistics-v2.pdf
 
# select your variable; name must be a valid ERA5 CDS API name.
var = "2m_temperature"
 
# Select the required statistic, valid names given in link above
stat = "daily_mean"
 
# Loop over years and months
 
for yr in years:
    for mn in months:
        result = c.service(
        "tool.toolbox.orchestrator.workflow",
        params={
             "realm": "user-apps",
             "project": "app-c3s-daily-era5-statistics",
             "version": "master",
             "kwargs": {
                 "dataset": "reanalysis-era5-single-levels",
                 "product_type": "reanalysis",
                 "variable": var,
                 "statistic": stat,
                 "year": yr,
                 "month": mn,
                 "time_zone": "UTC+00:0",
                 "frequency": "1-hourly",
#
# Users can change the output grid resolution and selected area
#
#                "grid": "1.0/1.0",
#                "area":{"lat": [10, 60], "lon": [65, 140]}
 
                 },
        "workflow_name": "application"
        })
         
# set name of output file for each month (statistic, variable, year, month     
 
        file_name = "download_" + stat + "_" + var + "_" + yr + "_" + mn + ".nc"
         
        location=result[0]['location']
        res = requests.get(location, stream = True)
        print("Writing data to " + file_name)
        with open(file_name,'wb') as fh:
            for r in res.iter_content(chunk_size = 1024):
                fh.write(r)
        fh.close()

'''